<a href="https://colab.research.google.com/github/sarahouses/proyecto_telecom/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Librerias

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from typing import Callable, List, Dict, Tuple
from pathlib import Path
from pandas import json_normalize

from IPython.display import clear_output

#Paths

In [49]:
data_path = Path('src')

if not data_path.exists():
    data_path.mkdir(parents=True)  # 'parents=True' crea directorios intermedios si es necesario
    print("Directorio creado.")
else:
    print("El directorio ya existe.")

El directorio ya existe.


#📌 Extracción

In [55]:
#URL donde se encuentran los datos
url = 'https://raw.githubusercontent.com/sarahouses/proyecto_telecom/refs/heads/main/TelecomX_Data%20(1).json'
response = requests.get(url)
data = response.json()
#Escribir el archivo para normalización
with open(data_path/'TelecomX_Data.json', 'wb') as f:
   f.write(response.content)
#Leer el archivo
with open(data_path/'TelecomX_Data.json', 'r') as f:
    data = json.loads(f.read())

In [62]:
data
df = pd.DataFrame(data)
df.head()


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [57]:
# Crear copia para mantener datos en su estado original por si se necesitan
df = data.copy()

**customerID:** número de identificación único de cada cliente

**Churn:** si el cliente dejó o no la empresa

**gender:** género (masculino y femenino)

**SeniorCitizen**: información sobre si un cliente tiene o no una edad igual o mayor a 65 años

**Partner:** si el cliente tiene o no una pareja

**Dependents:** si el cliente tiene o no dependientes

**tenure**: meses de contrato del cliente

**PhoneService:** suscripción al servicio telefónico

**MultipleLines:** suscripción a más de una línea telefónica

**InternetService:** suscripción a un proveedor de internet

**OnlineSecurity:** suscripción adicional de seguridad en línea

**OnlineBackup:** suscripción adicional de respaldo en línea

**DeviceProtection:** suscripción adicional de protección del dispositivo

**TechSupport:** suscripción adicional de soporte técnico, menor tiempo de espera

**StreamingTV:** suscripción de televisión por cable

**StreamingMovies:** suscripción de streaming de películas

**Contract:** tipo de contrato

**PaperlessBilling:** si el cliente prefiere recibir la factura en línea

**PaymentMethod:** forma de pago

**Charges.Monthly:** total de todos los servicios del cliente por mes

**Charges.Total:** total gastado por el cliente

In [64]:
type(df)

pandas.core.frame.DataFrame

In [65]:
for col in df.columns.tolist():
    if df[col].apply(lambda x: isinstance(x, dict)).any():
        normalized_data = pd.json_normalize(df[col])
        df = df.drop(columns=[col]).join(normalized_data)

print(df.head())

   customerID Churn  gender  SeniorCitizen Partner Dependents  tenure  \
0  0002-ORFBO    No  Female              0     Yes        Yes       9   
1  0003-MKNFE    No    Male              0      No         No       9   
2  0004-TLHLJ   Yes    Male              0      No         No       4   
3  0011-IGKFF   Yes    Male              1     Yes         No      13   
4  0013-EXCHZ   Yes  Female              1     Yes         No       3   

  PhoneService MultipleLines InternetService  ... OnlineBackup  \
0          Yes            No             DSL  ...          Yes   
1          Yes           Yes             DSL  ...           No   
2          Yes            No     Fiber optic  ...           No   
3          Yes            No     Fiber optic  ...          Yes   
4          Yes            No     Fiber optic  ...           No   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0               No         Yes         Yes              No        One year   
1       

#🔧 Transformación

In [ ]:
df.info()



In [ ]:
#DUPLICADOS

duplicados = df.duplicated()

# Imprimir las filas duplicadas
print(df[duplicados])

# Imprimir el número de filas duplicadas
print(f'Número de filas duplicadas: {duplicados.sum()}')


In [ ]:
#Valores unicos
for col in df.columns:
    # Encontrar los valores únicos en la columna
    valores_unicos = df[col].unique()

    # Imprimir los valores únicos
    print(f'Valores únicos en la columna {col}: {valores_unicos}')

In [ ]:
#Valores nulos
# Verificar si hay valores nulos en el DataFrame
hay_valores_nulos = df.isnull().any().any()

# Imprimir el resultado
if hay_valores_nulos:
    print('Hay valores nulos en el DataFrame')
else:
    print('No hay valores nulos en el DataFrame')

# Verificar si hay valores nulos en cada columna
valores_nulos_por_columna = df.isnull().any()

# Imprimir el resultado
print('Valores nulos por columna:')
print(valores_nulos_por_columna)

# Eliminar filas con valores nulos
df = df.dropna()
df

In [ ]:
#Valores en blanco o vacíos
import numpy as np

# Reemplazar cadenas vacías y cadenas que contienen solo espacios en blanco con NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Vacíos o en blanco
df.apply(lambda x: x.astype(str).str.strip() == '').sum()

# Verificar si hay valores nulos en el DataFrame
hay_valores_nulos = df.isnull().any().any()

# Imprimir el resultado
if hay_valores_nulos:
    print('Hay valores nulos (antes en blanco o vacíos) en el DataFrame')
else:
    print('No hay valores nulos (antes en blanco o vacíos) en el DataFrame')

# Verificar si hay valores nulos en cada columna
valores_nulos_por_columna = df.isnull().any()

# Imprimir el resultado
print('Valores nulos por columna:')
print(valores_nulos_por_columna)

In [ ]:
#Transformar datos

# Convertir las columnas 'Charges.Monthly' y 'Charges.Total' a tipo float y "tenure" a tipo float y "SeniorCitizen" a float
df['Charges.Monthly'] = df['Charges.Monthly'].astype(float)
df['Charges.Total'] = df['Charges.Total'].astype(float)
df['tenure']=df['tenure'].astype(float)
df['SeniorCitizen']=df['SeniorCitizen'].astype(float)

# Eliminar filas que contienen solo NaN
df = df.dropna(how='all')

#Vacios o en blanco
df.apply(lambda x: x.astype(str).str.strip() == '').sum()
# Verificar el tipo de datos de las columnas
print(df.dtypes)


In [ ]:


# Imprimir valores únicos en la columna 'Churn' antes del reemplazo
print("Valores únicos en 'Churn' antes del reemplazo:", df['Churn'].unique())

# Reemplazar cadenas vacías y cadenas que contienen solo espacios en blanco con NaN
df['Churn'] = df['Churn'].replace(r'^\s*$', np.nan, regex=True)

# Imprimir valores únicos en la columna 'Churn' después del reemplazo inicial
print("Valores únicos en 'Churn' después del reemplazo inicial:", df['Churn'].unique())

# Reemplazar explícitamente otros valores no reconocidos como NaN con NaN
df['Churn'] = df['Churn'].replace(['', ' '], np.nan)  # Añade aquí otros valores si los encuentras

# Eliminar filas con valores NaN en la columna 'Churn'
df = df.dropna(subset=['Churn'])

# Imprimir el número de filas y columnas del DataFrame después de quitar los valores NaN
print(f'Número de filas y columnas después de quitar los valores NaN: {df.shape}')

#📊 Carga y análisis

In [ ]:
df.describe()
df.head()

In [ ]:
#Análisis Descriptivo
df.info()
df.describe()

In [ ]:
#Distribución de evasión
import matplotlib.pyplot as plt
# Contar el número de clientes que se dieron de baja y los que permanecieron
churn_counts = df['Churn'].value_counts()

# Imprimir el número de clientes que se dieron de baja y los que permanecieron
print(churn_counts)

# Calcular la proporción de clientes que se dieron de baja y los que permanecieron
churn_proportions = df['Churn'].value_counts(normalize=True)

# Imprimir la proporción de clientes que se dieron de baja y los que permanecieron
print(churn_proportions)

# Crear un gráfico de barras para visualizar la distribución de la variable 'Churn'
churn_counts.plot(kind='bar')
plt.title('Distribución de la variable Churn')
plt.xlabel('Churn')
plt.ylabel('Número de clientes')
plt.show()

# Crear un gráfico de torta para visualizar la proporción de clientes que se dieron de baja y los que permanecieron
churn_proportions.plot(kind='pie', autopct='%1.1f%%')
plt.title('Proporción de clientes que se dieron de baja y los que permanecieron')
plt.ylabel('')
plt.show()

In [ ]:
#Recuento de evasión por variables categóricas
# Lista de variables categóricas a analizar
categorical_features = ['gender', 'Contract', 'PaymentMethod']

# Bucle para analizar cada variable categórica
for feature in categorical_features:
    # Crear tabla de contingencia
    contingency_table = pd.crosstab(df[feature], df['Churn'])

    # Calcular la proporción de cada categoría de la variable categórica
    proportions = contingency_table.div(contingency_table.sum(axis=1), axis=0)

    # Crear gráfico de barras apiladas
    proportions.plot(kind='bar', stacked=True)
    plt.title(f'Distribución de Churn por {feature}')
    plt.xlabel(feature)
    plt.ylabel('Proporción de clientes')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
#Recuento de evasión por variables numéricas
import seaborn as sns
# Lista de variables numéricas a analizar
numerical_features = ['Charges.Monthly', 'Charges.Total',"tenure"]

# Bucle para analizar cada variable numérica
for feature in numerical_features:
    # Crear histogramas
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=feature, hue='Churn', kde=True)
    plt.title(f'Distribución de {feature} por Churn')
    plt.show()

    # Crear diagramas de caja
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=df, x='Churn', y=feature)
    plt.title(f'Boxplot de {feature} por Churn')
    plt.show()



In [ ]:
# Calcular estadísticas descriptivas por grupo
descriptive_stats = df.groupby('Churn')['tenure'].describe()
print(f'Estadísticas descriptivas para tenure por Churn:\n{descriptive_stats}')

#📄Informe final


### **Introducción**
El churn representa la tasa de abandono de clientes. Entender sus causas es clave para mejorar la retención, reducir costos de adquisición y fortalecer la fidelidad del cliente. El objetivo de este análisis es identificar las variables asociadas a la pérdida de clientes y proponer estrategias de retención.
###**Limpieza y tratamiento de los datos**
Se importaron los datos con pandas y se identificaron valores nulos en la columna TotalCharges, los cuales fueron eliminados. Se convirtió TotalCharges a numérico y se codificaron variables categóricas donde fue necesario.
Librerías utilizadas:
- `pandas` para manipulación de datos.
- `numpy` para operaciones estadísticas.
- `matplotlib` y `seaborn` para la visualización.
### Analisis, Exploración de los datos, Conclusiones y Recomendaciones
### **Por lo que se puede visualizar:**
-**En cuanto a la distribución general de Chrun**,más de 1 de cada 4 clientes abandona, lo cual es un porcentaje relevante para considerar acciones correctivas o preventivas.
-**El generó no es una variable diferenciadora**, ya que ambos géneros presentan tasas similares de abandono y permanencia


-**Por metodó de pago**:Los métodos automáticos (transferencia bancaria y tarjeta de crédito) tienen una menor proporción de churn, mientras que el cheque electrónico presenta la mayor tasa de abandono (Gráfico 1).Lo cual podría indicar que los métodos automáticos fomentan una mayor permanencia por comodidad o menor fricción.

-**Por tipo de contrato:** El churn se concentra fuertemente en clientes con contratos month-to-month (Gráfico 3), donde la proporción de abandono es visiblemente mayor. En cambio, los contratos de un año y dos años tienen tasas de churn considerablemente menores, lo que sugiere que los contratos más largos ayudan a retener clientes. *Como recomendación, sería incentivar contratos de mayor duración podría reducir la tasa de churn.*

-**Relación entre Churn y Tenure (antigüedad del cliente)**
Los clientes que no desertaron (Churn = No) tienen un tenure promedio de 37.6 meses. En cambio, los que sí desertaron (Churn = Yes) tienen un promedio mucho menor: 18 meses.
El boxplot muestra claramente que los clientes con menor antigüedad tienen más probabilidad de irse. EN cuanto a la distribución:La mayoría de los abandonos se concentran entre clientes con menos de 20 meses de permanencia.Por ende se concluyé que, la antigüedad es un fuerte predictor del churn. Los clientes nuevos son mucho más propensos a abandonar. Como recomendación sería *implementar programas de fidelización en los primeros 24 meses de servicio para reducir la pérdida temprana.*

-**Relación entre Churn y Cargos Totales (Charges.Total)**
El Boxplot nos muestra que: Los clientes que se quedaron (No) tienden a tener cargos totales más altos, lo cual está alineado con su mayor tiempo de permanencia. En cuanto a la distribución: El churn es más frecuente entre clientes con bajo cargo total, lo que refuerza la idea de que no duraron mucho tiempo como clientes, así que se puede concluir que hay una relación indirecta entre churn y cargos acumulados; los clientes que abandonan no llegan a acumular altos cargos por su menor tiempo de permanencia.

-**Relación entre Churn y Cargos Mensuales (Charges.Monthly)**
El Boxplot y en cuanto a la distribución: En este caso, los clientes que se dieron de baja (Yes) tienen un promedio mensual más alto de cargos que los que se quedan.Lo cual,esto sugiere que precios altos mensuales pueden generar insatisfacción o que quienes contratan servicios más caros abandonan más rápido si no perciben valor.Se pude concluir que, a diferencia del cargo total, los altos cargos mensuales se asocian a mayor churn, posiblemente por percepciones negativas de costo-beneficio. *Se recomienda monitorear satisfacción en clientes con planes más costosos y ofrecer beneficios extra o atención personalizada.*

**Como conclusión englobada:**

Los clientes más propensos a abandonar:

-Tienen contratos mensuales.

-Pagan mediante cheques electrónicos.

-Han estado menos de 20 meses en la empresa.

-Tienen altos cargos mensuales, pero bajo cargo acumulado.

*Es pertinente comentar que el género no es un factor determinante*.

